In [ ]:
from tqdm import tqdm
from ibapi.client import EClient
from ibapi.common import BarData
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract

import threading
import time

class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.df = []
        self.df_list = []
        self.count = 0
        self.data_event = threading.Event()

    def historicalData(self, reqId, bar):
        op = bar.open
        hi = bar.high
        lo = bar.low
        cl = bar.close
        vol = bar.volume
        da = bar.date
        self.df.append([da, op, hi, lo, cl, vol])

    def historicalDataEnd(self, reqId: int, start: str, end: str):
        super().historicalDataEnd(reqId, start, end)  # Call base class implementation
        self.df_list.append(self.df)  # Add collected data to list
        self.df = []  # Clear the dataframe for the next request
        self.count += 1  # Increment the count of completed requests
        self.data_event.set()  # Signal that data retrieval is complete

def run_loop():
    app.run()
    
app = IBapi()
app.connect('127.0.0.1', 7496, 111)
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()
time.sleep(1)

for symbol in ['NVDA', "AAPL", "AMD"]:
	print(symbol)
	apple_contract = Contract()
	apple_contract.symbol = symbol
	apple_contract.secType = 'STK'
	apple_contract.exchange = 'SMART'
	apple_contract.currency = 'USD'
	print(apple_contract)
	#Request Market Data
	app.reqHistoricalData(
		reqId=1,
		contract=apple_contract, 
		endDateTime='', 
		durationStr='1 D', 
		barSizeSetting='5 mins', 
		whatToShow='BID', 
		useRTH=True, 
		formatDate=1, 
		keepUpToDate=False, 
		chartOptions=[])
	print(app.df)
app.disconnect()
